In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install -U transformers accelerate
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer("Where is Beirut?", return_tensors="pt")

input_ids = inputs['input_ids'].to("cuda")

attention_mask = inputs['attention_mask'].to("cuda")
print(input_ids)
print(attention_mask)

tensor([[128000,   9241,    374,  95411,     30]], device='cuda:0')
tensor([[1, 1, 1, 1, 1]], device='cuda:0')


In [ ]:
model.to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)
outputs

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[128000,   9241,    374,  ...,   2134,    315,  57088]],
       device='cuda:0')

In [ ]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

'Where is Beirut? Beirut is the capital and largest city of Lebanon, a small country located in the Middle East. It is situated on the eastern shore of the Mediterranean Sea, approximately 35 kilometers (22 miles) north of the Israeli-Lebanese border.\nWhat is the climate like in Beirut? Beirut has a Mediterranean climate, characterized by mild winters and hot, dry summers. The city\'s proximity to the Mediterranean Sea moderates its climate, with temperatures ranging from 10°C (50°F) in winter to 30°C (86°F) in summer. The city experiences a significant amount of rainfall, with an average annual rainfall of around 700 mm (28 in).\nWhat is the history of Beirut? Beirut has a rich and storied history, dating back over 5,000 years. The city was an important center of trade and commerce in the ancient world, and was known as the "Phoenician city" due to its connections with the Phoenicians, a seafaring people who inhabited the eastern Mediterranean. The city was later conquered by the Rom

In [ ]:
inputs = tokenizer("أين توجد مدينة بيروت؟", return_tensors="pt")

input_ids = inputs['input_ids'].to("cuda")

attention_mask = inputs['attention_mask'].to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'أين توجد مدينة بيروت؟ Where is the city of Beirut?\nBeirut is the capital and largest city of Lebanon, located on the Mediterranean coast. It is situated in the western part of the country, about 35 kilometers (22 miles) north of the border with Israel. Beirut is a major city in the Middle East, known for its rich history, cultural attractions, and vibrant nightlife.\nBeirut is located on a narrow coastal plain, surrounded by mountains to the east and the sea to the west. The city has a population of around 360,000 people, and it is the economic and commercial center of Lebanon. Beirut is also home to many international organizations, universities, and cultural institutions.\nBeirut is a city with a rich history, dating back to the Phoenician era. It was an important center of trade and commerce in the ancient world, and it was later conquered by the Romans, Arabs, and Ottomans. The city has been shaped by its strategic location, and it has played a significant role in the history of 

In [15]:
inputs = tokenizer("أجب بالعربي: أين توجد مدينة بيروت؟", return_tensors="pt")

input_ids = inputs['input_ids'].to("cuda")

attention_mask = inputs['attention_mask'].to("cuda")

outputs = model.generate(input_ids, attention_mask=attention_mask)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'أجب بالعربي: أين توجد مدينة بيروت؟\n\nEnglish translation: Answer in Arabic: Where is the city of Beirut located?\n\nAnswer: بيروت هي مدينة ساحلية في لبنان، وتقع على ساحل البحر الأبيض المتوسط. (Beirut is a coastal city in Lebanon, located on the Mediterranean coast.)'

In [16]:
SYS_PROMPT = """You are an assistant for answering questions.
 Only give the answer
 Dont add any comments or extra explanations.
 If you don't know the answer, just say "I do not know." Don't make up an answer.
 Answer in Arabic if the question is in Arabic
 """

In [17]:
def format_prompt(question, context=""):
    PROMPT = f"My Question is : {question}\n"
    if context!="":
      PROMPT = PROMPT   + f" You have to Answer in the following Context: {context}\n"
    return PROMPT

In [27]:
def talk(question, context=""):
    formatted_prompt  = format_prompt(question, context)
    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": formatted_prompt }
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")


    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

    outputs_ids = model.generate(
        input_ids=input_ids,
        pad_token_id=pad_token_id,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.01,
        top_p=0.95
    )

    output_text = tokenizer.decode(outputs_ids[0], skip_special_tokens=True)

    return output_text

In [25]:
def get_last_line(s):
    lines = s.splitlines()
    return lines[-1] if lines else ''

In [28]:
question = "What is Python?"

result = talk(question)
last_line = get_last_line(result)

print(last_line)

Python is a high-level, interpreted programming language.


In [29]:
question = "ما هي بايثون؟"

result = talk(question)
last_line = get_last_line(result)

print(last_line)

بايثون (Python) لغة برمجة عالية المستوى، متعددة الاستخدامات، وسهلة التعلم، تستخدم في تطوير تطبيقات الكمبيوتر والأنظمة والويب والهوات


In [30]:
question = "What is Apple?"

result = talk(question)
last_line = get_last_line(result)

print(last_line)

Apple is a multinational technology company that designs, manufactures, and markets consumer electronics, computer software, and online services.


In [31]:
question = "What is Apple? "

context= "Tree and Fruits"
result = talk(question, context)
last_line = get_last_line(result)

print(last_line)

Apple is a type of fruit that grows on an apple tree.


In [32]:
question = " بم تشتهر مدينة بيروت "

result = talk(question)
last_line = get_last_line(result)

print(last_line)

تشتهر مدينة بيروت بالثقافة والفنون والسياحة والاقتصاد والشوارع التاريخية والبحر الأزرق.


In [33]:
question = "هل تقع بيروت على شاطئ البحر"

result = talk(question)
last_line = get_last_line(result)

print(last_line)

نعم، تقع بيروت على شاطئ البحر.


In [34]:
question = " Where is the 4Z Hotel?"

result = talk(question)
last_line = get_last_line(result)

print(last_line)

I do not know.
